In [1]:
import pandas as pd

from sqlalchemy import create_engine

In [2]:
path_to_db = '/datasets/telecomm_csi.db'
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

In [3]:
query = """
SELECT u.user_id,
    u.lt_day,
    CASE 
        WHEN u.lt_day > 365 THEN 'старый'
        ELSE 'новый'
    END AS is_new,
    u.age,
    CASE 
        WHEN u.gender_segment = 1 THEN 'женщина'
        WHEN u.gender_segment = 0 THEN 'мужчина'
        ELSE 'unknown'
    END AS gender_segment,
    u.os_name,
    u.cpe_type_name,
    l.country,
    l.city,
    substr(ag.title,4) AS age_segment,
    substr(ts.title,4) AS traffic_segment,
    CASE
        WHEN u.lt_day < 182 THEN '0-0.5'
        WHEN u.lt_day < 365 THEN '0.5-1'
        WHEN u.lt_day < 730 THEN '1-2'
        WHEN u.lt_day < 1095 THEN '2-3'
        WHEN u.lt_day < 1460 THEN '3-4'
        ELSE '5+' 
    END AS lifetime_segment,
    u.nps_score,
    CASE
        WHEN u.nps_score < 7 THEN 'критики'
        WHEN u.nps_score < 9 THEN 'нейтралы'
        ELSE 'cторонники'
    END AS nps_group
FROM user AS u
JOIN location AS l ON u.location_id=l.location_id
JOIN age_segment AS ag ON u.age_gr_id=ag.age_gr_id
JOIN traffic_segment AS ts ON u.tr_gr_id=ts.tr_gr_id
JOIN lifetime_segment AS ls ON u.lt_gr_id=ls.lt_gr_id;
"""

In [4]:
df = pd.read_sql(query, engine)
df

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,старый,45.0,женщина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,5+,10,cторонники
1,A001WF,2344,старый,53.0,мужчина,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,5+,10,cторонники
2,A003Q7,467,старый,57.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,1-2,10,cторонники
3,A004TB,4190,старый,44.0,женщина,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,5+,10,cторонники
4,A004XT,1163,старый,24.0,мужчина,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,3-4,10,cторонники
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502488,ZZZKLD,1249,старый,54.0,женщина,ANDROID,SMARTPHONE,Россия,Москва,45-54,1-5,3-4,5,критики
502489,ZZZLWY,129,новый,31.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,25-34,1-5,0-0.5,8,нейтралы
502490,ZZZQ5F,522,старый,36.0,мужчина,ANDROID,SMARTPHONE,Россия,Сургут,35-44,25-30,1-2,10,cторонники
502491,ZZZQ8E,2936,старый,37.0,женщина,ANDROID,SMARTPHONE,Россия,УланУдэ,35-44,65-70,5+,9,cторонники


In [5]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

### Ссылка на дашборд на сайте Tableau Public:

https://public.tableau.com/app/profile/woxtik/viz/NPS_16558807743650/Dashboard1